# Analyzing the NYC Subway Dataset

Project connected to the [Udacity Intro to Data Science course](https://www.udacity.com/course/viewer#!/c-ud359-nd).

by Victor Ribeiro, October/2015

---

### Section 0. References

**About the Dataset**

Turnstile and Weather Variables dataset reports on the cumulative number of entries and exists in the NYC with additional information about the weather. 

* [Original Dataset](https://www.dropbox.com/s/meyki2wl9xfa7yk/turnstile_data_master_with_weather.csv) - data set used throughout the course and used in the report below.
* [Improved Dataset](https://www.dropbox.com/s/1lpoeh2w6px4diu/improved-dataset.zip?dl=0) - cleaned-up subset of original dataset with additional variables. [Variables in the dataset](https://s3.amazonaws.com/uploads.hipchat.com/23756/665149/05bgLZqSsMycnkg/turnstile-weather-variables.pdf)

**References**

* [Mann-Whitney U Test](https://storage.googleapis.com/supplemental_media/udacityu/4332539257/MannWhitneyUTest.pdf) Udacity
* [Mann-Whitney U Test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) Wikipedia
* [Shapiro-Wilk Test](https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test) Wikipedia
* [Shapiro-Wild Test](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html) Python reference
* [Diez, David; Barr, Christopher; Çetinkaya-Rundel, Mine] OpenIntro Statistics, Third Edition

---

### Section 1. Statistical Test

**1.1 Which statistical test did you use to analyze the NYC subway data? Did you use a one-tail or a two-tail P value? What is the null hypothesis? What is your p-critical value?**

>Considering the proposed project :
>* Null hypothesis : there's no difference between number of rides in the metro during raining days vs. not raining days. 
>* Alternative hypothesis : there's a difference between number of rides in the metro during raining days vs.
not raining days.
>
>A Mann-Whitney U Test is applied. It's a two-tail test and p-critical value 5% (or 0.05).

---
**1.2 Why is this statistical test applicable to the dataset? In particular, consider the assumptions that the test is making about the distribution of ridership in the two samples.**

>Taking a look in the data to support decision on 1.1

In [1]:
import numpy as np
import pandas
import pandasql
import matplotlib.pyplot as plt
import datetime
import csv
import scipy
import scipy.stats
import statsmodels.api as sm
import sys
from ggplot import *
import itertools

df = pandas.read_csv("turnstile_data_master_with_weather.csv")
df.describe()

,Unnamed: 0,Hour,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,mindewpti,minpressurei,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
count,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951.000000,131951
mean,65975.000000,10.896158,1095.348478,886.890838,30.031894,57.241302,48.259013,29.892714,52.703526,29.965077,0.167100,0.334245,5.543065,56.169775,64.269729,71.769968,0.172276,0
std,38091.117022,6.892084,2337.015421,2008.604886,0.125689,8.770891,11.305312,0.146384,9.943590,0.130461,0.373066,0.471728,1.982441,6.338875,6.568289,7.627218,0.429005,0
min,0.000000,0.000000,0.000000,0.000000,29.740000,39.000000,22.000000,29.540000,31.000000,29.640000,0.000000,0.000000,1.000000,46.000000,55.000000,58.000000,0.000000,0
25%,32987.500000,5.000000,39.000000,32.000000,29.960000,50.000000,38.000000,29.840000,45.000000,29.910000,0.000000,0.000000,5.000000,52.000000,60.000000,65.000000,0.000000,0
50%,65975.000000,12.000000,279.000000,232.000000,30.030000,57.000000,51.000000,29.910000,54.000000,29.960000,0.000000,0.000000,5.000000,54.000000,63.000000,71.000000,0.000000,0
75%,98962.500000,17.000000,1109.000000,847.000000,30.100000,64.000000,55.000000,29.970000,60.000000,30.050000,0.000000,1.000000,6.000000,60.000000,68.000000,78.000000,0.100000,0
max,131950.000000,23.000000,51839.000000,45249.000000,30.310000,70.000000,66.000000,30.230000,68.000000,30.270000,1.000000,1.000000,12.000000,70.000000,78.000000,86.000000,2.180000,0


In [81]:
df.head()

,Unnamed: 0,UNIT,DATEn,TIMEn,Hour,DESCn,ENTRIESn_hourly,EXITSn_hourly,maxpressurei,maxdewpti,...,meandewpti,meanpressurei,fog,rain,meanwindspdi,mintempi,meantempi,maxtempi,precipi,thunder
0,0,R001,2011-05-01,01:00:00,1,REGULAR,0,0,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
1,1,R001,2011-05-01,05:00:00,5,REGULAR,217,553,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
2,2,R001,2011-05-01,09:00:00,9,REGULAR,890,1262,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
3,3,R001,2011-05-01,13:00:00,13,REGULAR,2451,3708,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0
4,4,R001,2011-05-01,17:00:00,17,REGULAR,4400,2501,30.31,42,...,39,30.27,0,0,5,50,60,69,0,0


In [2]:
plt.figure()
bins = 25
alpha = 0.75
df[df['rain']==0]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
df[df['rain']==1]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
    
plt.suptitle('Histogram of ENTRIESn_hourly')
plt.ylabel('Frequency')
plt.xlabel('ENTRIESn_hourly')
plt.legend(['no rain', 'rain'])
plt.grid(True)
plt.show()

![Histogram Raining and Non-Raining](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_1.png?raw=true)

>As per histogram above neither raining days nor no-raining days data follow a normal distribution. Indeed, by applying Shapiro-Wik test (below), we confirm datasets do not follow normal distribution as p-value for shapiro test on both raining / no-raining data is really small. 

In [3]:
print scipy.stats.shapiro(df[df['rain']==0]['ENTRIESn_hourly'])
print scipy.stats.shapiro(df[df['rain']==1]['ENTRIESn_hourly'])

(0.47661787271499634, 0.0)
(0.4715914726257324, 0.0)


/Users/vribeiro/anaconda/lib/python2.7/site-packages/scipy/stats/morestats.py:997: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


>Thus, a non-parametric test like Mann-Whithney U Test is applicable.

---
**1.3 What results did you get from this statistical test? These should include the following numerical values: p-values, as well as the means for each of the two samples under test.**

>Applying Mann-Whitney U Test.



In [4]:
with_rain_mean = np.mean(df[df['rain']==1]['ENTRIESn_hourly'])
without_rain_mean = np.mean(df[df['rain']==0]['ENTRIESn_hourly'])
    
U,p = scipy.stats.mannwhitneyu(df[df['rain']==1]['ENTRIESn_hourly'],
                               df[df['rain']==0]['ENTRIESn_hourly'])

print with_rain_mean, without_rain_mean, U, 2*p

1105.44637675 1090.27878015 1924409167.0 0.049999825587


**1.4 What is the significance and interpretation of these results?**

>`ENTRIESn_hourly` raining mean is slightly bigger than no-raining means - which makes sense as people may prefer walk if it's not raining (personally, I would guess a much higher difference).2 \* p-value is slightly below 0.05, thus **null hypothesis is rejected**.

---


### Section 2. Linear Regression

**2.1 What approach did you use to compute the coefficients theta and produce prediction for ENTRIESn_hourly in your regression model:**
- **OLS using Statsmodels or Scikit Learn,**
- **Gradient descent using Scikit Learn,**
- **Or something different?**

>OLS (using Statsmodels) has been selected to predict `ENTRIESn_hourly`.

In [92]:
def linear_regression(features, values):
    model = sm.OLS(values,sm.add_constant(features))
    results = model.fit()
    intercept = results.params[0]
    params = results.params[1:]    
    
    return intercept, params

features = df[['Hour','rain','meantempi','minpressurei']] 
dummy_units = pandas.get_dummies(df['UNIT'], prefix='unit')
features = features.join(dummy_units)

# Perform linear regression
intercept, params = linear_regression(features, df['ENTRIESn_hourly'])
    
predictions = intercept + np.dot(features, params)

def compute_r_squared(data, predictions):
    n = ((data - predictions)**2).sum()
    d = ((data - data.mean())**2).sum()
    
    r_squared = 1 - n/d
    return r_squared

compute_r_squared(df['ENTRIESn_hourly'], predictions)


0.45845668581696875

In [106]:
# collection with almost all features. Following Ex 5 on Set 3, EXITSn_hourly is not being used.
aall_features = ['rain','fog', 'precipi', 'Hour', 
                 'meanwindspdi',
                 'maxtempi', 'mintempi', 'meantempi', 
                 'maxpressurei', 'minpressurei', 'meanpressurei',
                 'maxdewpti', 'mindewpti', 'meandewpti' ]

# multiple variables to log results
i = 0
t_feat = []
t_inte = []
t_para = []
t_pred = []
t_rsqu = []
t_subs = []
log_experiments = []

# global max logs and counter
r_max = -1
s_max = 'none'
j = 0

# This brute force loop will select all combinations of some specific sizes. 
# At first, not using dummies variables.
for L in range(4,6): #(len(aall_features)+1)):
    log_experiments.append(j)
    r_max_local = -1
    
    # for each combination, runs a linear regression, loging data, preserving max
    for subset in itertools.combinations(aall_features, L):
        t_feat.append(i)
        t_inte.append(i)
        t_para.append(i)
        t_pred.append(i)
        t_rsqu.append(i)
        t_subs.append(i)
        
        t_subs[i] = subset
        
        # Selected features
        t_feat[i] = df[[subset[0]]]
        for k in range(1,len(subset)):
            t_feat[i] = t_feat[i].join(df[[subset[k]]])
            
        # Perform linear regression
        t_inte[i], t_para[i] = linear_regression(t_feat[i], df['ENTRIESn_hourly'])
        t_pred[i] = t_inte[i] + np.dot(t_feat[i], t_para[i])
        t_rsqu[i] = compute_r_squared(df['ENTRIESn_hourly'], t_pred[i])
        
        # Saving max for each combination size
        if r_max_local < t_rsqu[i]:
            r_max_local = t_rsqu[i]
            log_experiments[j] = [r_max_local, t_subs[i], i]
            
        #print 'Test ',i,' Subset:', subset, 'R2:', t_rsqu[i]
        i = i+1
    
    # Saving total max for all combinations size so far
    if r_max < r_max_local:
        r_max = r_max_local
        s_max = log_experiments[j]
        
    j = j+1
        
# print Rˆ2 max for each combination size and features used
for k in range(0,len(log_experiments)):
    print log_experiments[k][0], log_experiments[k][1], log_experiments[k][2]

KeyboardInterrupt: 

In [91]:
# now for the best combinations without dummies, add dummies variables
log_experiments_dummies = []

j = 0
for k in log_experiments:
    log_experiments_dummies.append(j)
    
    subset = log_experiments[j][1]
    
    t_feat.append(i)
    t_inte.append(i)
    t_para.append(i)
    t_pred.append(i)
    t_rsqu.append(i)
    t_subs.append(i)
        
    t_subs[i] = subset
        
    # Selected features + dummies
    t_feat[i] = df[[subset[0]]]
    for k in range(1,len(subset)):
        t_feat[i] = t_feat[i].join(df[[subset[k]]])
            
    t_feat[i] = t_feat[i].join(pandas.get_dummies(df['UNIT'], prefix='unit'))
        
    # Perform linear regression
    t_inte[i], t_para[i] = linear_regression(t_feat[i], df['ENTRIESn_hourly'])
    t_pred[i] = t_inte[i] + np.dot(t_feat[i], t_para[i])
    t_rsqu[i] = compute_r_squared(df['ENTRIESn_hourly'], t_pred[i])
      
    log_experiments_dummies[j] = [t_rsqu[i], t_subs[i], i]
            
    # Saving global max so far
    if r_max < t_rsqu[i]:
        r_max = t_rsqu[i]
        s_max = log_experiments_dummies[j]    

    print 'Test ',i,' Subset:', subset, 'R2:', t_rsqu[i]
    i = i+1        
    j = j+1

    
# print Rˆ2 max for each subset with dummies and features used
for k in range(0,len(log_experiments_dummies)):
    print log_experiments_dummies[k][0], log_experiments_dummies[k][1], '+ Dummies : UNIT'

Test  3003  Subset: ('Hour', 'mintempi', 'maxdewpti', 'meandewpti') R2: 0.459513261048
Test  3004  Subset: ('Hour', 'maxtempi', 'meantempi', 'maxpressurei', 'maxdewpti') R2: 0.459755485495
0.459513261048 ('Hour', 'mintempi', 'maxdewpti', 'meandewpti') + Dummies : UNIT and hour
0.459755485495 ('Hour', 'maxtempi', 'meantempi', 'maxpressurei', 'maxdewpti') + Dummies : UNIT and hour


---
**2.2 What features (input variables) did you use in your model? Did you use any dummy variables as part of your features?**

>Selected variables were : `UNIT` as dummy variable and :

In [107]:
print s_max

none



---
**2.3 Why did you select these features in your model? We are looking for specific reasons that lead you to believe that the selected features will contribute to the predictive power of your model.**
- **Your reasons might be based on intuition. For example, response for fog might be: “I decided to use fog because I thought that when it is very foggy outside people might decide to use the subway more often.”**
- **Your reasons might also be based on data exploration and experimentation, for example: “I used feature X because as soon as I included it in my model, it drastically improved my R2 value.”**

>As I lived majority of my live in a city without metro, I do not have any strong gut feeling about what are the variables that would influentiated the most. Indeed, I've tried different combinations and results were really different one from each other. Thus, I've decided for some brute force : (i) pick all combinations with 4 and 5 elements and calculate R2 without a dummy variable and then, pick the best results and try with a dummy variable.
>
>The dummy variable 'UNIT' drastically improves R2 value. Max R2 value was `0.459755485495` for features mentioned in 2.2.

---

**2.4 What are the parameters (also known as "coefficients" or "weights") of the non-dummy features in your linear regression model?**

In [99]:
t_para[3004].head(5)

Hour             67.424637
maxtempi         47.498094
meantempi       -71.922292
maxpressurei   -424.532240
maxdewpti        12.536417
dtype: float64

---
**2.5 What is your model’s R2 (coefficients of determination) value?**

In [100]:
compute_r_squared(df['ENTRIESn_hourly'], t_pred[3004])

0.4597554854950926

---

**2.6 What does this R2 value mean for the goodness of fit for your regression model? Do you think this linear model to predict ridership is appropriate for this dataset, given this R2  value?**

> R2 is the the percentage of variance that is explained. Max R2 obtained explains only 46% of variation, which is low. Additional study may lead to best results. For instance, more features or polynomial regressions plus regularization to mitigate over-fitting.

---


### Section 3. Visualization

**Please include two visualizations that show the relationships between two or more variables in the NYC subway data.
Remember to add appropriate titles and axes labels to your plots. Also, please add a short description below each figure commenting on the key insights depicted in the figure.**

**3.1 One visualization should contain two histograms: one of  ENTRIESn_hourly for rainy days and one of ENTRIESn_hourly for non-rainy days.**
- **You can combine the two histograms in a single plot or you can use two separate plots.**
- **If you decide to use to two separate plots for the two histograms, please ensure that the x-axis limits for both of the plots are identical. It is much easier to compare the two in that case.**
- **For the histograms, you should have intervals representing the volume of ridership (value of ENTRIESn_hourly) on the x-axis and the frequency of occurrence on the y-axis. For example, each interval (along the x-axis), the height of the bar for this interval will represent the number of records (rows in our data) that have ENTRIESn_hourly that falls in this interval.**
- **Remember to increase the number of bins in the histogram (by having larger number of bars). The default bin width is not sufficient to capture the variability in the two samples.**


In [104]:
plt.figure()
bins = 20
alpha = 0.50
df[df['rain']==0]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
df[df['rain']==1]['ENTRIESn_hourly'].hist(bins = bins, alpha=alpha) 
    
plt.suptitle('Histogram of ENTRIESn_hourly')
plt.ylabel('Frequency')
plt.xlabel('ENTRIESn_hourly')
plt.legend(['no rain', 'rain'])
plt.grid(True)
plt.show()

![Histogram Raining and Non-Raining](https://github.com/vfribeiro/IntroDataScience/blob/master/figure_1.png?raw=true)

---

**3.2 One visualization can be more freeform. You should feel free to implement something that we discussed in class (e.g., scatter plots, line plots) or attempt to implement something more advanced if you'd like. Some suggestions are:**
- **Ridership by time-of-day**
- **Ridership by day-of-week**

---

### Section 4. Conclusion

**Please address the following questions in detail. Your answers should be 1-2 paragraphs long.**

**4.1 From your analysis and interpretation of the data, do more people ride
the NYC subway when it is raining or when it is not raining?**

>Yes, according the study here presented, it's possible to conclude with a high level of condifence that more people ride the NYC subway when it's raining than when it's not raining.

**4.2 What analyses lead you to this conclusion? You should use results from both your statistical
tests and your linear regression to support your analysis.**

>In section 1, a full statistical analysis was presented. First, data was analyzed to verify what kind of statistical test could be used. Then, after selecting running Mann-Whitney U Test, obtained p-value leaded to reject the null hypothesis stating no difference between rides when it's raining vs it's not raining.



---

### Section 5. Reflection

**Please address the following questions in detail. Your answers should be 1-2 paragraphs long.**

**5.1 Please discuss potential shortcomings of the methods of your analysis, including:**
- **Dataset,**
- **Analysis, such as the linear regression model or statistical test.**

**5.2 (Optional) Do you have any other insight about the dataset that you would like to share with us?**

---